In [1]:
from keras.models import load_model
import cv2
import numpy as np
from random import choice

In [2]:
REV_CLASS_MAP = {
    0:"face_mask",
    1:"no_face",
    2:"no_mask",
}

In [3]:
def mapper(val):
    return REV_CLASS_MAP[val]

In [4]:
REV_CLASS_MAP[1]

'no_face'

In [5]:
model = load_model("face_mask_detection.h5")

In [12]:

cap = cv2.VideoCapture(0)

prev_move = None

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # rectangle for user to play
    cv2.rectangle(frame, (100, 100), (400, 400), (255, 255, 255), 2)
    # rectangle for computer to play
#     cv2.rectangle(frame, (350, 0), (550, 250), (255, 255, 255), 2)

    # extract the region of image within the user rectangle
    roi = frame[100:400, 100:400]
    roi = roi/255.0
#     img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, (227, 227))

    # predict the move made
    pred = model.predict(np.array([roi]))
    move_code = np.argmax(pred[0])
#     print(move_code)
    user_move_name = mapper(move_code)

    # predict the winner (human vs computer)
    if prev_move != user_move_name:
#         if user_move_name != "none":
        computer_move_name = user_move_name
        
    prev_move = user_move_name

    # display the information
    font = cv2.FONT_HERSHEY_SIMPLEX

    cv2.putText(frame, "Face Mask Detection: " + computer_move_name,
                (150, 450), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
#     cv2.putText(frame, "Winner: " + winner,
#                 (300, 300), font, 0.8, (0, 0, 255), 4, cv2.LINE_AA)

#     if computer_move_name != "none":
#         icon = cv2.imread(
#             "images/{}.png".format(computer_move_name))
#         icon = cv2.resize(icon, (200, 200))
#         frame[0:200, 350:550] = icon

    cv2.imshow("Face Mask Detector:", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()